In [1]:
from __future__ import absolute_import, division, print_function

import os
import pathlib
import re
import struct

# set GPU to invisible
# os.environ['CUDA_VISIBLE_DEVICES'] = ''
# os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image

import tensorflow as tf
from tensorflow import keras

print(tf.VERSION)
tf.enable_eager_execution()

AUTOTUNE = tf.data.experimental.AUTOTUNE
_BUFFER_SIZE = 69137
_BATCH_SIZE = 128
INPUT_SHAPE = (64, 64)

1.13.1


## Prepare the datasets

### Prepare the data as `numpy` array from packed data

In [2]:
def numpy_etl1(etl1_path):
    
    datasets = []
    labels = []
    
    def unpack_dataset(path):
        img_arr = []
        label_arr = []
        
        f = open(path, 'rb')
        while True:
            s = f.read(2052)

            if not len(s) == 2052:
                print('[{}] Reach EOF, remain {} bytes unread.'.format(path, len(s)))
                break;
            record = struct.unpack('>H2sH6BI4H4B4x2016s4x', s)
            # label at index 1
            label = record[1].decode('ascii')

            if ' ' in label: # remove spaces
                label = label.replace(' ', '')
            # image at index 18
            iF = Image.frombytes('F', (64, 63), record[18], 'bit', 4)
            np_img = np.array(iF, dtype=np.uint8) # np_img.shape = (63, 64)
            np_img = cv2.resize(np_img, INPUT_SHAPE, interpolation=cv2.INTER_CUBIC)
            np_img.reshape((1, INPUT_SHAPE[0], INPUT_SHAPE[1]))
            scaled_img = np_img.astype(np.float) / 15.0
            
            img_arr.append(scaled_img)
            label_arr.append(label)

        f.close()
        
        return img_arr, label_arr
    
    # we only need the katakana characters
    skip_etls = [
        'ETL1C_01',
        'ETL1C_02',
        'ETL1C_03',
        'ETL1C_04',
        'ETL1C_05',
        'ETL1C_06',
    ]
    if not os.path.exists(etl1_path):
        print('Invalid path')
        return None
    
    etl1_files = os.listdir(etl1_path)
    
    for etl in etl1_files:
        if not re.search('ETL1C_\d\d', etl) == None:
            if etl in skip_etls:
                continue
            etl_path = os.path.join(etl1_path, etl)
            
            img_arr, label_arr = unpack_dataset(etl_path)
            datasets.extend(img_arr)
            labels.extend(label_arr)
            
    return (np.array(datasets), np.array(labels))

### Prepare the datasets as `tf.data.Dataset` from unpacked data

In [ ]:
def compile_etl_datasets():
    data_root = pathlib.Path('./etlcb_01_datasets/')
    all_image_paths = list(data_root.glob('*/*'))
    all_image_paths = [str(path) for path in all_image_paths]

    label_names = sorted(
        item.name for item in data_root.glob('*/') if item.is_dir())
    label_to_index = dict((name, index)
                         for index, name in enumerate(label_names))

    all_image_labels = [label_to_index[pathlib.Path(path).parent.name]
                        for path in all_image_paths]

    def process_image(img):
        img = tf.image.decode_png(img, channels=1)
        img = tf.image.resize_images(img, [64, 64])

        img /= 255.0
        return img

    def load_and_process_image(path):
        img = tf.read_file(path)
        return process_image(img)

    path_ds = tf.data.Dataset.from_tensor_slices(all_image_paths)
    img_ds = path_ds.map(load_and_process_image, num_parallel_calls=AUTOTUNE)

    label_ds = tf.data.Dataset.from_tensor_slices(
        tf.cast(all_image_labels, tf.int64))

    return img_ds, label_ds

## Keras model

In [ ]:
def create_kana_cnn_model():
    input_shape=(64, 64, 1, )
    model = keras.Sequential([
        keras.layers.Conv2D(filters=32, 
                            kernel_size=5,
                            padding='same',
                            activation=tf.nn.relu,
                            input_shape=input_shape, 
                            data_format='channels_last', 
                            name='kana_conv1'),
        keras.layers.MaxPooling2D(pool_size=2, 
                                  strides=2),
        keras.layers.Conv2D(filters=64, 
                            kernel_size=5, 
                            padding='same', 
                            activation=tf.nn.relu, 
                            name='kana_conv2'),
        keras.layers.MaxPooling2D(pool_size=2, 
                                  strides=2),
        keras.layers.Flatten(),
        keras.layers.Dense(1024, activation=tf.nn.relu),
        keras.layers.Dense(46, activation=tf.nn.softmax)
    ])
    return model

### Another keras model

In [ ]:
def kana_keras_v2():
    model = keras.Sequential([
        keras.layers.Conv2D(
            filters=32, 
            kernel_size=5, 
#             strides=2, 
            padding='same',
            activation=tf.nn.relu,
            input_shape=(64,64,1,), 
            data_format='channels_last'
            ),
        keras.layers.MaxPool2D(
            pool_size=2, 
#             strides=(2, 2),
            # padding='same'
            ),
        keras.layers.Conv2D(
            filters=16, 
            kernel_size=5, 
#             strides=(2, 2), 
            padding='same',
            activation=tf.nn.relu,
            ),
        keras.layers.MaxPool2D(
            pool_size=(2, 2),
#             strides=(2, 2), 
            # padding='same'
            ),
        keras.layers.Conv2D(
            filters=16, 
            kernel_size=3, 
            padding='same',
            activation=tf.nn.relu,
            ),
        keras.layers.MaxPool2D(
            pool_size=2, 
#             strides=(2, 2)
            ),
        keras.layers.Flatten(),
        keras.layers.Dense(
            units=128, 
            activation=tf.nn.relu,
            ),
        keras.layers.Dense(
            units=46,
            activation=tf.nn.softmax
        )
    ])
    return model
model = kana_keras_v2()
model.summary()

In [ ]:
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

In [ ]:
train_data, train_label = compile_etl_datasets()

In [ ]:
kana_ds = tf.data.Dataset.zip((train_data, train_label))
kana_ds.apply(tf.data.experimental.shuffle_and_repeat(buffer_size=_BUFFER_SIZE))
kana_ds.batch(_BATCH_SIZE)
kana_ds.prefetch(buffer_size=AUTOTUNE)

In [ ]:
print('output_shapes:', kana_ds.output_shapes)
print('output_classes:', kana_ds.output_classes)
print('output_types:', kana_ds.output_types)

In [ ]:
model.fit(kana_ds, epochs=1, steps_per_epoch=5)

In [ ]:
# img_ds = tf.data.Dataset.zip((train_data, train_label))
img_ds = train_data

In [ ]:
print(img_ds.output_classes)
print(img_ds.output_shapes)
print(img_ds.output_types)

In [ ]:
IMG_COUNT = 69137
BATCH_SIZE = 32

In [ ]:
img_ds.cache().apply(tf.data.experimental.shuffle_and_repeat(buffer_size=IMG_COUNT))
img_ds.batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)

### Failed to feed data into the Keras model

In [ ]:
model.fit(x=train_data, y=train_label, epochs=5, steps_per_epoch=IMG_COUNT)

## TensorFlow Estimator model

In [ ]:
def kana_estimator_fn(features, output):
    

In [3]:
datasets, labels = numpy_etl1('./ETL1/')

[./ETL1/ETL1C_07] Reach EOF, remain 0 bytes unread.
[./ETL1/ETL1C_08] Reach EOF, remain 0 bytes unread.
[./ETL1/ETL1C_09] Reach EOF, remain 0 bytes unread.
[./ETL1/ETL1C_10] Reach EOF, remain 0 bytes unread.
[./ETL1/ETL1C_11] Reach EOF, remain 0 bytes unread.
[./ETL1/ETL1C_12] Reach EOF, remain 0 bytes unread.
[./ETL1/ETL1C_13] Reach EOF, remain 0 bytes unread.


In [4]:
print(datasets.shape)
print(labels.shape)

(71959, 64, 64)
(71959,)


In [ ]:
idx = 60
plt.imshow(datasets[idx])
plt.title(labels[idx])
plt.colorbar()